# Web Scraping - Lab

## Introduction

Now that you've seen a more extensive example of developing a web scraping script, it's time to further practice and formalize that knowledge by writing functions to parse specific pieces of information from the web page and then synthesizing these into a larger loop that will iterate over successive web pages in order to build a complete dataset.

## Objectives

You will be able to:

* Navigate HTML documents using Beautiful Soup's children and sibling relations
* Select specific elements from HTML using Beautiful Soup
* Use regular expressions to extract items with a certain pattern within Beautiful Soup
* Determine the pagination scheme of a website and scrape multiple pages

## Lab Overview

This lab will build upon the previous lesson. In the end, you'll look to write a script that will iterate over all of the pages for the demo site and extract the title, price, star rating and availability of each book listed. Building up to that, you'll formalize the concepts from the lesson by writing functions that will extract a list of each of these features for each web page. You'll then combine these functions into the full script which will look something like this:  

```python
df = pd.DataFrame()
for i in range(2,51):
    url = "http://books.toscrape.com/catalogue/page-{}.html".format(i)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    new_titles = retrieve_titles(soup)
    new_star_ratings = retrieve_ratings(soup)
    new_prices = retrieve_prices(soup)
    new_avails = retrieve_avails(soup)
    ...
 ```

## Import Test Data

In [42]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
html_page = requests.get('http://books.toscrape.com/')
soup = BeautifulSoup(html_page.content, 'html.parser')

In [9]:
def get_books(soup):
    books_container = soup.find('ol', class_="row")
    books = books_container.findAll('article', class_="product_pod")
    return books

In [11]:
books = get_books(soup)
book = books[0]
book

<article class="product_pod">
<div class="image_container">
<a href="catalogue/a-light-in-the-attic_1000/index.html"><img alt="A Light in the Attic" class="thumbnail" src="media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg"/></a>
</div>
<p class="star-rating Three">
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
</p>
<h3><a href="catalogue/a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a></h3>
<div class="product_price">
<p class="price_color">£51.77</p>
<p class="instock availability">
<i class="icon-ok"></i>
    
        In stock
    
</p>
<form>
<button class="btn btn-primary btn-block" data-loading-text="Adding..." type="submit">Add to basket</button>
</form>
</div>
</article>

## Retrieving Titles

To start, write a function that extracts the titles of the books on a given page. The input for the function should be the `soup` for the HTML of the page.

In [10]:
def get_title(book):
    title = book.find('h3').find('a').attrs['title']
    return title

In [12]:
get_title(book)

'A Light in the Attic'

## Retrieve Ratings

Next, write a similar function to retrieve the star ratings on a given page. Again, the function should take in the `soup` from the given HTML page and return a list of the star ratings for the books. These star ratings should be formatted as integers.

In [17]:
def get_rating(book):
    regex = re.compile("star-rating (.*)")
    star_rating_str = book.find('p', {"class" : regex}).attrs['class'][-1]
    star_dict = {'One': 1, 'Two': 2, 'Three':3, 'Four': 4, 'Five':5}
    star_rating = star_dict[star_rating_str]
    return star_rating

In [18]:

get_rating(book)

3

## Retrieve Prices

Now write a function to retrieve the prices on a given page. The function should take in the `soup` from the given page and return a list of prices formatted as floats.

In [25]:
def get_price(book):
    price = float(book.find('p', class_="price_color").text[1:])
    return price
    

In [26]:
get_price(book)

51.77

## Retrieve Availability

Write a function to retrieve whether each book is available or not. The function should take in the `soup` from a given html page and return a list of the availability for each book.

In [37]:
def get_availability(book):
    availability = book.find('p', class_="instock availability").text.strip()
    return availability

In [38]:
get_availability(book)

'In stock'

## Make Book Dictionary

In [40]:
def make_book_dict(book):
    book_dict = {
        'Title': get_title(book),
        'Star_Rating': get_rating(book),
        'Price': get_price(book),
        'Availablility': get_availability(book)
    }
    return book_dict

In [41]:
make_book_dict(book)

{'Title': 'A Light in the Attic',
 'Star_Rating': 3,
 'Price': 51.77,
 'Availablility': 'In stock'}

## Create a Script to Retrieve All the Books From All 50 Pages

Finally, write a script to retrieve all of the information from all 50 pages of the books.toscrape.com website. 

In [43]:
books_list = []
for i in range(2,51):
    url = "http://books.toscrape.com/catalogue/page-{}.html".format(i)
    html_page = requests.get(url)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    books = get_books(soup)
    for book in books:
        books_list.append(make_book_dict(book))
df = pd.DataFrame(books_list)
df.head()

,Title,Star_Rating,Price,Availablility
0,In Her Wake,1,12.84,In stock
1,How Music Works,2,37.32,In stock
2,Foolproof Preserving: A Guide to Small Batch J...,3,30.52,In stock
3,Chase Me (Paris Nights #2),5,25.27,In stock
4,Black Dust,5,34.53,In stock


In [44]:
len(df)

980

## Level-Up: Write a new version of the script you just wrote. 

If you used URL hacking to generate each successive page URL, instead write a function that retrieves the link from the `"next"` button at the bottom of the page. Conversely, if you already used this approach above, use URL-hacking (arguably the easier of the two methods in this case).

In [ ]:
#Your code here

## Summary

Well done! You just completed your first full web scraping project! You're ready to start harnessing the power of the web!